final outputs:

Digital boundary (AUS range, manually select VIC if needed)
- ../data/raw/ABS/digitalBoundary/SA2_2021_AUST_GDA2020.shp

SA2 code to district names
- ../data/raw/ABS/SA2_TO_Name.csv
- code(int), name(String)


Estimated Resident Population (ERP) (2001 to 2021) (By SA2)
- ../data/raw/ABS/ERP/ERP.csv
- SA2 (int), year (int), population (int)
- SA2: interger code marking each district, e.g. 206041117 for Carlton

Household income (weekly) (exclude visitor/non-classifiable) (2021) (By SA2)
- ../data/raw/ABS/Household_income/Household_income.csv
- SA2 (int), year (int),  household_type (String), income_level (String), popultaion (int)

Population projection (2017 - 2066) (VIC overall)
- ../data/raw/ABS/Population/Population.csv
- year (int), popultaion (int)

School location
- ../data/raw/ABS/School_location/School_location.csv
- School Name (String), SA2 (int), Latitude (float), Longitude (float), School Type (String)



In [1]:
import os
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import re
from itertools import compress


spark = (
    SparkSession.builder.appName("Assignment_2")
    .config("spark.sql.repl.eagerEval.enabled", True)
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

INPUT_DIR = "../data/raw/ABS/"

22/09/06 20:50:27 WARN Utils: Your hostname, Bruce-PC resolves to a loopback address: 127.0.1.1; using 172.21.200.124 instead (on interface eth0)
22/09/06 20:50:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/06 20:50:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/09/06 20:50:30 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Read all dataframe

In [2]:
# read school file
school_sdf = spark.read.csv(f"{INPUT_DIR}School_location/School_location.csv", header=True)

# count school by SA2
school_count = school_sdf.groupBy("SA2").agg({
    "School Name": "count"
})
school_count = school_count.withColumnRenamed( "count(School Name)", "school_count")
school_count.show(1, vertical = True, truncate=100)
school_count.createOrReplaceTempView("school")

-RECORD 0-----------------
 SA2          | 209031212 
 school_count | 9         
only showing top 1 row



In [3]:
# read ERP file and create tempview
ERP_sdf = spark.read.csv(f"{INPUT_DIR}ERP/ERP.csv", header=True)
ERP_sdf.show(1, vertical = True, truncate=100)
ERP_sdf.createOrReplaceTempView("ERP")

-RECORD 0---------------
 SA2        | 201011481 
 year       | 2010      
 population | 8664      
only showing top 1 row



In [4]:
# read median household income file and create tempview
household_sdf = spark.read.csv(f"{INPUT_DIR}Household_income/Household_income.csv", header=True)
household_sdf.show(2, vertical = True, truncate=100)
household_sdf.createOrReplaceTempView("household")

-RECORD 0------------------
 SA2           | 213051589 
 median_income | 1862      
-RECORD 1------------------
 SA2           | 209041437 
 median_income | 1979      
only showing top 2 rows



In [7]:
# inner join
print(school_count.columns)
print(ERP_sdf.columns)
print(household_sdf.columns)
combine_sdf = spark.sql("""
SELECT  school.SA2, ERP.year, school.school_count, ERP.population AS ERP_population, median_income
FROM school
INNER JOIN ERP ON school.SA2 = ERP.SA2
INNER JOIN household ON school.SA2 = household.SA2
""")
combine_sdf.show(1, vertical = True, truncate=100)


['SA2', 'school_count']
['SA2', 'year', 'population']
['SA2', 'median_income']
-RECORD 0-------------------
 SA2            | 202011018 
 year           | 2010      
 school_count   | 13        
 ERP_population | 14752     
 median_income  | 1267      
only showing top 1 row



In [12]:
print(sorted( combine_sdf.select(F.col('year')).distinct().select("year").rdd.flatMap(lambda x: x).collect() ))

['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021']
